<center><h1>US Domestic Airline Flights Performance</h1></center>

In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
from dash.exceptions import PreventUpdate
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

In [4]:
 # needs to be run again in a separate cell due to a jupyterdash bug
JupyterDash.infer_jupyter_proxy_config()

In [5]:
external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]


report_type_options = [
    {"label": "Yearly airline performance report", "value": "yap"},
    {"label": "Yearly average flight delay statistics", "value": "yads"},
]

year_options = [{"label": x, "value": x} for x in range(2005, 2021)]



# Read the airline data into pandas dataframe
airline_data =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})

""" Compute_info function description

This function takes in airline data and selected year as an input and performs computation for creating charts and plots.

Arguments:
    airline_data: Input airline data.
    entered_year: Input year for which computation needs to be performed.
    
Returns:
    Computed average dataframes for carrier delay, weather delay, NAS delay, security delay, and late aircraft delay.

"""
def compute_info(airline_data, entered_year):
    # Select data
    df =  airline_data[airline_data['Year']==int(entered_year)]
    # Compute delay averages
    avg_car = df.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late
    

In [6]:
print(str(report_type_options[0]["value"]))

yap


In [25]:
# Create a dash application
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    [   html.H1('Choose Parameter', style={'textAlign': 'center', 'color': '#503D36', 'font-size': 30}),
        html.Label(["Report type", dcc.Dropdown(id="report_type", options=report_type_options)]),
        html.Label(["Choose Year", dcc.Dropdown(id="year", options=year_options)]),
        html.Div(dcc.Graph(id='tree-plot')),
        html.Br(),
        html.Div([
                html.Div(dcc.Graph(id='pie-plot')),
                html.Div(dcc.Graph(id='map-plot'))
        ], style={'display': 'flex'}),
        html.Br(),
        html.Div([
                html.Div(dcc.Graph(id='bar-plot')),
                html.Div(dcc.Graph(id='line-plot'))
        ], style={'display': 'flex'}),

        html.Br(), 
        html.Div([
                html.Div(dcc.Graph(id='carrier-plot')),
                html.Div(dcc.Graph(id='weather-plot'))
        ], style={'display': 'flex'}),

        html.Div([
                html.Div(dcc.Graph(id='nas-plot')),
                html.Div(dcc.Graph(id='security-plot'))
        ], style={'display': 'flex'}),

        html.Div(dcc.Graph(id='late-plot'), style={'display': 'flex'})
    ]
)

# Callback hiding
@app.callback(
    Output("tree-plot", "style"),
    Output("pie-plot", "style"),
    Output("map-plot", "style"),
    Output("bar-plot", "style"),
    Output("line-plot", "style"),
    Input("report_type", "value"),
    Input("year", "value"))
def display_chart_yap(report_type, year):
    if not report_type or not year:
        style = [{"display": "none"} for x in range(5)]
    else:
        style = [{"display": "flex"} for x in range(5)]
    return style

# Callback hiding
@app.callback(
    Output("carrier-plot", "style"),
    Output("weather-plot", "style"),
    Output("nas-plot", "style"),
    Output("security-plot", "style"),
    Output("late-plot", "style"),
    Input("report_type", "value"),
    Input("year", "value"))
def display_chart_yads(report_type, year):
    if not report_type or not year:
        style = [{"display": "none"} for x in range(5)]
    else:
        style = [{"display": "flex"} for x in range(5)]
    return style

# Callback decorator
@app.callback(
    Output("tree-plot", "figure"),
    Output("pie-plot", "figure"),
    Output("map-plot", "figure"),
    Output("bar-plot", "figure"),
    Output("line-plot", "figure"),
    Input("report_type", "value"),
    Input("year", "value"))
def get_graph_yap(report_type, year):
    if not report_type or not year:
        raise PreventUpdate
    elif report_type == report_type_options[0]["value"]:
        df =  airline_data[airline_data['Year']==int(year)]

        # Group the data by Month and compute average over arrival delay time.
        bar_data = df.groupby(['Reporting_Airline'])['ArrDelay'].sum().reset_index()
        line_data = df.groupby('Month')['ArrDelay'].mean().reset_index()
        map_data = df.groupby('DestState')['ArrDelay'].mean().reset_index()

        tree_fig = px.treemap(df, path=['Month', 'Reporting_Airline'], values='Flights')

        pie_fig = px.pie(df, values='ArrDelay', names='Reporting_Airline', title='US Average Flight Delay by Airline')

        map_fig = go.Figure(data=go.Choropleth(
            locations=map_data['DestState'], # Spatial coordinates
            z = map_data['ArrDelay'].astype(float), # Data to be color-coded
            locationmode = 'USA-states', # set of locations match entries in `locations`
            colorscale = 'Reds',
            colorbar_title = "Delay MIN",
        ))

        map_fig.update_layout(
            title_text = 'US Average Flight Delay by State',
            geo_scope='usa', # limite map scope to USA
        )

        bar_fig = px.bar(bar_data, x="Reporting_Airline", y="ArrDelay", title='Average Flight Delay split by reporting airline') 

        line_fig = go.Figure(data=go.Scatter(x=line_data['Month'], y=line_data['ArrDelay'], mode='lines', marker=dict(color='green')))
        line_fig.update_layout(title='Month vs Average Flight Delay Time', xaxis_title='Month', yaxis_title='ArrDelay')

        return [tree_fig, pie_fig, map_fig, bar_fig, line_fig]
    
    else: return[{"display": "none"} for x in range(5)]
    
# Callback decorator
@app.callback(
    Output("carrier-plot", "figure"),
    Output("weather-plot", "figure"),
    Output("nas-plot", "figure"),
    Output("security-plot", "figure"),
    Output("late-plot", "figure"),
    Input("report_type", "value"),
    Input("year", "value"))
def get_graph_yads(report_type, year):
    if not report_type or not year:
        raise PreventUpdate
    elif report_type == report_type_options[1]["value"]:
        # Compute required information for creating graph from the data
        avg_car, avg_weather, avg_NAS, avg_sec, avg_late = compute_info(airline_data, year)

        # Create graph
        carrier_fig = px.line(avg_car, x='Month', y='CarrierDelay', color='Reporting_Airline', title='Average carrrier delay time (minutes) by airline')
        weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay', color='Reporting_Airline', title='Average weather delay time (minutes) by airline')
        nas_fig = px.line(avg_NAS, x='Month', y='NASDelay', color='Reporting_Airline', title='Average NAS delay time (minutes) by airline')
        sec_fig = px.line(avg_sec, x='Month', y='SecurityDelay', color='Reporting_Airline', title='Average security delay time (minutes) by airline')
        late_fig = px.line(avg_late, x='Month', y='LateAircraftDelay', color='Reporting_Airline', title='Average late aircraft delay time (minutes) by airline')

        return [carrier_fig, weather_fig, nas_fig, sec_fig, late_fig]
    
    else: return[{"display": "none"} for x in range(5)]


if __name__ == "__main__":
    app.run_server(mode="inline", host="localhost", debug=True)
    